In [3]:
import cv2
import numpy as np
import dlib
from imutils import face_utils
import time

# Cargar modelo entrenado
face_recognizer = cv2.face.LBPHFaceRecognizer_create()
face_recognizer.read('modeloLBPHFace.xml')

# Cargar múltiples clasificadores
face_frontal = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
face_alt = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
face_perfil = cv2.CascadeClassifier('haarcascade_profileface.xml')

# Cargar detector y predictor de dlib para puntos de referencia faciales
detector_dlib = dlib.get_frontal_face_detector()
predictor_dlib = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

# Definir nombres de usuarios según etiquetas usadas en el entrenamiento
usuarios = ["Marcelo", "Samuel", "Yoana"]

# Umbral de reconocimiento
umbral_reconocimiento = 70
tiempo_estatico = 7.0  # Tiempo en segundos para considerar una imagen estática
tiempo_inicial_parpadeo = time.time()

# Inicializar captura de video
cap = cv2.VideoCapture(0)

# Variables para detectar imágenes estáticas
rostro_anterior = None
contador_estatico = 0  # Contador para rostros estáticos

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    auxFrame = frame.copy()

    # Detectar rostros con múltiples clasificadores
    detected_faces = []
    for face_list in [
        face_frontal.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=6, minSize=(40, 40)),
        face_alt.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=6, minSize=(40, 40)),
        face_perfil.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5, minSize=(50, 50)),
        face_perfil.detectMultiScale(cv2.flip(gray, 1), scaleFactor=1.3, minNeighbors=5, minSize=(50, 50))
    ]:
        if len(face_list) > 0:
            detected_faces.extend(face_list)

    for (x, y, w, h) in detected_faces:
        cv2.rectangle(frame, (x, y), (x + w, y + h), (128, 0, 255), 2)
        rostro = auxFrame[y:y + h, x:x + w]
        rostro_gray = cv2.cvtColor(cv2.resize(rostro, (150, 150)), cv2.COLOR_BGR2GRAY)

        # **Verificar si el rostro es estático**
        if rostro_anterior is not None:
            diferencia = np.sum(np.abs(rostro_anterior - rostro_gray))
            if diferencia < 600:  # Ajusta este umbral según pruebas
                contador_estatico += 1
            else:
                contador_estatico = 0  # Reiniciar si hay movimiento

        tiempo_actual_parpadeo = time.time()
        rects = detector_dlib(gray, 0)

        for rect in rects:
            shape = predictor_dlib(gray, rect)
            shape = face_utils.shape_to_np(shape)

            # **Dibujar puntos faciales**
            for (sx, sy) in shape:
                cv2.circle(frame, (sx, sy), 2, (0, 255, 0), -1)

            # **Detectar parpadeo antes de reconocer**
            ojo_izquierdo = shape[36:42]
            ojo_derecho = shape[42:48]
            ratio_izquierdo = np.linalg.norm(ojo_izquierdo[1] - ojo_izquierdo[5]) / np.linalg.norm(ojo_izquierdo[0] - ojo_izquierdo[3])
            ratio_derecho = np.linalg.norm(ojo_derecho[1] - ojo_derecho[5]) / np.linalg.norm(ojo_derecho[0] - ojo_derecho[3])
            ratio_parpadeo = (ratio_izquierdo + ratio_derecho) / 2

            if ratio_parpadeo < 0.18:  # Si detecta parpadeo, confirma que es un usuario real
                tiempo_inicial_parpadeo = time.time()
                id_usuario, confianza = face_recognizer.predict(rostro_gray)

                if confianza < umbral_reconocimiento and id_usuario < len(usuarios):
                    texto = f"{usuarios[id_usuario]} - Confianza: {confianza:.2f}"
                    color = (0, 255, 0)  # Verde → Identificado
                else:
                    texto = "Desconocido"
                    color = (0, 0, 255)  # Rojo → No registrado
            else:
                if tiempo_actual_parpadeo - tiempo_inicial_parpadeo >= tiempo_estatico:
                    texto = "Imagen estática detectada"
                    color = (0, 128, 255)  # Naranja
                else:
                    id_usuario, confianza = face_recognizer.predict(rostro_gray)

                    if confianza < umbral_reconocimiento and id_usuario < len(usuarios):
                        texto = f"{usuarios[id_usuario]} - Confianza: {confianza:.2f}"
                        color = (0, 255, 0)  # Verde → Identificado
                    else:
                        texto = "Desconocido"
                        color = (0, 0, 255)  # Rojo → No registrado

        # Almacenar el rostro actual para la próxima iteración
        rostro_anterior = rostro_gray.copy()

        cv2.putText(frame, texto, (x, max(y - 15, 30)), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2, cv2.LINE_AA)

    cv2.imshow('Reconocimiento Facial', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
